# Full Network Model


---


Make a network that has:


1. 6 Convolution layers with these kernels (10, 10, 20, 20, 30)
2. no fully connected layer (you probably may have to use the Global Average Pooling layer)
3. uses EMNIST as the dataset
4. uses a maximum of 2 max-pooling layers



In [94]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [95]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,10, 3, padding=0) # Input => 28 ; Output => 26 ; Receptive Field => 3 
  
        self.conv2 = nn.Conv2d(10,10, 3, padding=0) # Input => 26 ; Output => 24 ; Receptive Field => 5
        
        self.pool1 = nn.MaxPool2d(2, 2) # Input => 24 ; Output => 12 ; Receptive Field => 10

        self.conv3 = nn.Conv2d(10, 20, 3, padding=0) # Input => 12 ; Output => 10 ; Receptive Field => 12
        
        self.conv4 = nn.Conv2d(20, 20, 3, padding=0) # Input => 10 ; Output => 8 ; Receptive Field => 14
       
        self.conv5 = nn.Conv2d(20, 30, 3, padding=0) # Input => 8 ; Output => 6 ; Receptive Field => 16
       
        self.pool2 = nn.MaxPool2d(2, 2) # Input => 6 ; Output => 3 ; Receptive Field => 32

        self.conv6 = nn.Conv2d(30, 62, 3) # Input => 3 ; Output => 1 ; Receptive Field => 35
       
        self.aap = nn.AdaptiveAvgPool2d((5,1)) # Global Average pooling

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = F.relu(self.conv4(F.relu(self.conv3(x))))
        x = self.pool2((F.relu(self.conv5(x))))
        x = self.conv6(x)
        x = self.aap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [96]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 26, 26]             100
            Conv2d-2           [-1, 10, 24, 24]             910
         MaxPool2d-3           [-1, 10, 12, 12]               0
            Conv2d-4           [-1, 20, 10, 10]           1,820
            Conv2d-5             [-1, 20, 8, 8]           3,620
            Conv2d-6             [-1, 30, 6, 6]           5,430
         MaxPool2d-7             [-1, 30, 3, 3]               0
            Conv2d-8             [-1, 62, 1, 1]          16,802
 AdaptiveAvgPool2d-9             [-1, 62, 5, 1]               0
Total params: 28,682
Trainable params: 28,682
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.14
Params size (MB): 0.11
Estimated Total Size (MB): 0.26
---------------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
